<a href="https://colab.research.google.com/github/Jdmolano2020/TesisNer/blob/main/SroieTextAugmentation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Mount Google drive to colab
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import locale
def getpreferredencoding(do_setlocale = True):
    return "UTF-8"
locale.getpreferredencoding = getpreferredencoding

##Cargar Archivos Sroie

In [3]:
# limpiamos rastros del dataset y descomprimimos
!rm -rf /tmp/SROIE2019v02
!unzip /content/drive/MyDrive/archive_SROIEv2.zip -d /tmp/SROIE2019v02/

Archive:  /content/drive/MyDrive/archive_SROIEv2.zip
   creating: /tmp/SROIE2019v02/SROIE2019/layoutlm-base-uncased/
  inflating: /tmp/SROIE2019v02/SROIE2019/layoutlm-base-uncased/config.json  
  inflating: /tmp/SROIE2019v02/SROIE2019/layoutlm-base-uncased/pytorch_model.bin  
  inflating: /tmp/SROIE2019v02/SROIE2019/layoutlm-base-uncased/special_tokens_map.json  
  inflating: /tmp/SROIE2019v02/SROIE2019/layoutlm-base-uncased/tokenizer_config.json  
  inflating: /tmp/SROIE2019v02/SROIE2019/layoutlm-base-uncased/training_args.bin  
  inflating: /tmp/SROIE2019v02/SROIE2019/layoutlm-base-uncased/vocab.txt  
   creating: /tmp/SROIE2019v02/SROIE2019/test/
   creating: /tmp/SROIE2019v02/SROIE2019/test/box/
  inflating: /tmp/SROIE2019v02/SROIE2019/test/box/X00016469670.txt  
  inflating: /tmp/SROIE2019v02/SROIE2019/test/box/X00016469671.txt  
  inflating: /tmp/SROIE2019v02/SROIE2019/test/box/X51005200931.txt  
  inflating: /tmp/SROIE2019v02/SROIE2019/test/box/X51005230605.txt  
  inflating: /t

##Importar Librerias

In [4]:
# Instale la versión más reciente de gensim.
# De lo contrario, puede recibir el siguiente error al ejecutar naw.WordEmbsAug():
# El objeto 'Word2VecKeyedVectors' no tiene atributo 'index_to_key'
# see: https://stackoverflow.com/questions/71032760/word2veckeyedvectors-object-has-no-attribute-index-to-key
!pip install --upgrade gensim --quiet

In [5]:
# Importar gensim.
# Nota: deberá reiniciar el tiempo de ejecución para importar la versión más reciente de gensim
import gensim
print(gensim.__version__)

4.3.2


In [6]:
# Instale el tokenizador que necesita el modelo de traducción inversa
!pip install sacremoses --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 897.5/897.5 kB 7.6 MB/s eta 0:00:00


In [7]:
# Install the tokenizer
import sacremoses

In [8]:
# Install the nlpaug module
!pip install nlpaug --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.5/410.5 kB 6.4 MB/s eta 0:00:00


In [9]:
# Importar el módulo nlpaug y sus métodos
import nlpaug.augmenter.word as naw
import nlpaug.flow as nafc
from nlpaug.util import Action

In [10]:
# Install Huggingface Transformers
! pip install transformers[sentencepiece] --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 10.6 MB/s eta 0:00:00


In [11]:
# Install datasets
! pip install datasets --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 507.1/507.1 kB 5.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 8.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 7.8 MB/s eta 0:00:00


In [12]:
! pip install accelerate -U

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 265.7/265.7 kB 5.2 MB/s eta 0:00:00


In [13]:
# Import other modules
import nltk
nltk.download('wordnet')
nltk.download('stopwords')
nltk.download('omw-1.4')
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datasets import DatasetDict, Dataset
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from transformers import Trainer, TrainingArguments
from transformers import Trainer
from sklearn.metrics import accuracy_score, f1_score
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
import gc

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


In [14]:
import json
import random
import re
import joblib
from tqdm import tqdm

import spacy.cli
from spacy import displacy
from spacy.util import minibatch, compounding
from spacy.training import Example
from spacy.scorer import Scorer
import pathlib

from spacy.training import offsets_to_biluo_tags
from spacy.tokenizer import Tokenizer

## Crear Modelo NER

In [15]:
nlp = spacy.blank("en")
ner = nlp.create_pipe("ner")
nlp.add_pipe("ner")

In [16]:
def parse(line):
    fields = line.strip().split(",")
    if len(fields) == 9:
        return fields
    else:
        return fields[:8] + [",".join(fields[8:])]

def build_text(data):
    text = " ".join(data.text)
    text = text.replace("  "," ")
    return text

def entidades_total_prueba (ruta,archivo):
  #ruta ='/tmp/SROIE2019v02/SROIE2019/train/'
  #archivo = 'X51007419197.txt'

  print(archivo)
  with open(ruta + "box/" + archivo, encoding='utf-8', errors='ignore') as f:
    ocr = f.readlines()
  data = pd.DataFrame(
    list(map(parse, ocr)),
    columns=[*(f"coor{i}" for i in range(8)), "text"]
    )
  data=data.dropna()
  #print("data",data)
  texto = build_text(data)
  #print("texto",texto)

  with open(ruta + "entities/" + archivo, encoding='utf-8') as f:
    entities = json.load(f)
  #print("entities=",entities)
  #print("type_entities=",type(entities))
  entidades = []
  for name, ent_text in entities.items():
    if len(ent_text)==0:
      continue
    else:
      ent_text=ent_text.translate(str.maketrans({"-":  r"\-",
                                                  "]":  r"\]",
                                                  "\\": r"\\",
                                                  "^":  r"\^",
                                                  "$":  r"\$",
                                                  "*":  r"\*",
                                                  ".":  r"\.",
                                                  "(":  r"\(",
                                                  ")":  r"\)" }))
    #print("ent_text=",ent_text)
    matches = re.finditer(ent_text, texto)
    local_ent = map(lambda match: (*match.span(), name), matches)
    entidades.extend(list(local_ent))
  entidades.sort()

  label_iden = {"total": False,
                "date": False,
                "address": False,
                "company": False
      }
  #print("entidades",entidades)
  entidades_n = []
  for ent in reversed(entidades):
    #print("ent=",ent)
    label= ent[2]
    if label_iden[label]==False:
      label_iden[label]=True
      entidades_n.append(ent)
    #print("entidades_n=",entidades_n)
  entidades_n.sort()
  #print("entidades_n",entidades_n)
  texto_faltantes=[]
  i=0
  pos_ini = 0
  pos_fin = 0

  for ent in entidades_n:
    if (i == 0) & (ent[0] > 0):
      pos_ini = 0
      pos_fin = ent[0]-1
      texto_faltantes.append((pos_ini,pos_fin,texto[pos_ini:pos_fin]))
      #print("if 1 ent=",ent,"pos_ini=",pos_ini,"pos_fin=",pos_fin,"texto_f=",texto[pos_ini:pos_fin])
    if i < len(entidades_n)-1:
      if ent[1]+1 != entidades_n[i+1][0]:
        pos_ini = entidades_n[i][1]+1
        pos_fin = entidades_n[i+1][0]-1
        texto_faltantes.append((pos_ini,pos_fin,texto[pos_ini:pos_fin]))
        #print("if 2 ent=",ent,"pos_ini=",pos_ini,"pos_fin=",pos_fin,"texto_f=",texto[pos_ini:pos_fin])
    i+=1

  if (pos_fin < len(texto)-1) & (entidades_n[-1][1] < len(texto)-1):
    if pos_fin > entidades_n[-1][1]:
      pos_ini = pos_fin + 1
      pos_fin = len(texto)
    else:
      pos_ini = entidades_n[-1][1]+1
      pos_fin = len(texto)
    texto_faltantes.append((pos_ini,pos_fin,texto[pos_ini:pos_fin]))
    #print("if 3 ent=",ent, "pos_ini=",pos_ini,"pos_fin=",pos_fin,"texto_f=",texto[pos_ini:pos_fin])
  entidades_faltantes = []
  #print("texto_faltantes",texto_faltantes)
  for texto_fa in texto_faltantes:
    pos_ini,pos_fin,texto_f = texto_fa
    #print("pos_ini=",pos_ini," pos_fin=",pos_fin," texto_f=",texto_f)
    #tonkens = tokenizer(texto_f)
    tonkens = texto_f.split(' ')
    pos_ini_t = pos_ini
    for token in tonkens:
      pos_fin_t = pos_ini_t + len(token)
      #print("pos_ini_t=",pos_ini_t,"pos_fin_t=",pos_fin_t,"token=",token,"textoP=",texto[pos_ini_t:pos_fin_t])
      entidades_faltantes.append((pos_ini_t,pos_fin_t,'O'))
      pos_ini_t = pos_fin_t + 1
  #print("entidades_faltantes",entidades_faltantes)
  entidades_final=entidades_n + entidades_faltantes
  entidades_final.sort()
  spacy.training.offsets_to_biluo_tags(nlp.make_doc(texto), entidades_final)
  tags = offsets_to_biluo_tags(nlp.make_doc(texto), entidades_final)
  #print("tags=",tags)
  #doc_p = nlp.make_doc(texto)
  #for tok in doc_p:
   #print("tok=",tok)
  entidades_final=dict(entities=entidades_final)
  entidades_final= (texto,entidades_final,entidades_n)
  #print("entidades_final=",entidades_final)
  return(entidades_final)

In [17]:
def SacarTextoArchivo (ruta,archivo):
  #ruta ='/tmp/SROIE2019v02/SROIE2019/train/'
  #archivo = 'X51007419197.txt'

  print(archivo)
  with open(ruta + "box/" + archivo, encoding='utf-8', errors='ignore') as f:
    ocr = f.readlines()
  data = pd.DataFrame(
    list(map(parse, ocr)),
    columns=[*(f"coor{i}" for i in range(8)), "text"]
    )
  data=data.dropna()
  #print("data",data)
  texto = build_text(data)
  #print("texto",texto)
  texto_final = (texto,archivo)
  return(texto_final)

##Armar Dataset

In [18]:
data_dir = "/tmp/SROIE2019v02/SROIE2019/train/box"
data_dir = pathlib.Path(data_dir)
box_count = len(list(data_dir.glob('**/*.txt')))
print(box_count)
l_box=list(data_dir.glob('**/*.txt'))

data_dir = "/tmp/SROIE2019v02/SROIE2019/test/box"
data_dir = pathlib.Path(data_dir)
box_count_test = len(list(data_dir.glob('**/*.txt')))
print(box_count_test)
l_box_test=list(data_dir.glob('**/*.txt'))

626
347


In [19]:
l_box_archivo= list(map(lambda cadena: str(cadena)[-16:], l_box))
l_box_test_archivo= list(map(lambda cadena: str(cadena)[-16:], l_box_test))

In [20]:
textos_nerd = joblib.Parallel(n_jobs=-1)(joblib.delayed(SacarTextoArchivo)('/tmp/SROIE2019v02/SROIE2019/train/',archivo) for archivo in tqdm(l_box_archivo))

100%|██████████| 626/626 [00:03<00:00, 160.46it/s]


In [21]:
textos_nerd_test = joblib.Parallel(n_jobs=-1)(joblib.delayed(SacarTextoArchivo)('/tmp/SROIE2019v02/SROIE2019/test/',archivo) for archivo in tqdm(l_box_test_archivo))

100%|██████████| 347/347 [00:00<00:00, 351.32it/s]


In [22]:
df = pd.DataFrame(textos_nerd_test, columns=['text', 'archivo'])
test_df = df[['text', 'archivo']]
#print(test_df)
df = pd.DataFrame(textos_nerd, columns=['text',  'archivo'])
train_df = df[['text', 'archivo']]
#print(train_df)


In [23]:
# Convert the three DataFrames to three Datasets (Aparche Arrow format)
# Convierta los tres DataFrames de datos en tres conjuntos de datos (formato de flecha de Apache) (Aparche Arrow format)

dset_test = Dataset.from_pandas( test_df)
dset_train = Dataset.from_pandas(train_df)


In [24]:
# Gather train, val, and test Datasets to have a single DatasetDict, which can be manipulated together
# Reúna conjuntos de datos de train, val, y test para tener un solo DatasetDict, que se puede manipular juntos
sroie = DatasetDict({
  'train': dset_train,
  'test': dset_test})
# Dataset.from_pandas will add an index column, which can be removed
# Dataset.from_pandas agregará una columna de índice, que se puede eliminar

In [25]:
sroie


DatasetDict({
    train: Dataset({
        features: ['text', 'archivo'],
        num_rows: 626
    })
    test: Dataset({
        features: ['text', 'archivo'],
        num_rows: 347
    })
})

In [26]:
# Use GPU (cuda) if available
import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

## Initiate three alternative text augmentation strategies

In [27]:
# Initiate the synonym augmentation - # Iniciar el aumento de sinónimos
aug_syn = naw.SynonymAug(
  aug_src = 'wordnet',
  aug_max = 3)

In [28]:
augmented_sroie = []
augmented_sorie_archivo = []
for i in train_df.index:
    lst_augment = aug_syn.augment(train_df['text'].loc[i], n = 1)
    for augment in lst_augment:
        augmented_sroie.append(augment)
        augmented_sorie_archivo.append(train_df['archivo'].loc[i])

  # Zip the two lists into a list of tuples -  Comprima las dos listas en una lista de tuplas
augmented_sorie_textarchivo = list(zip(augmented_sroie, augmented_sorie_archivo))

df = pd.DataFrame(augmented_sorie_textarchivo, columns=['text', 'archivo'])
df_augmented = df[['text', 'archivo']]


[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


In [29]:
train_df_augmented = pd.concat([train_df, df_augmented], axis = 0)

In [30]:
train_df_augmented.describe()

,text,archivo
count,1252,1252
unique,1250,626
top,B & BEST RESTAURANT (001610235-P) GST NO. : 00...,X51007339642.txt
freq,2,2


In [31]:
# Initiate the contextual word embeddings (BERT, DistilBERT, RoBERTA or XLNet) augmentation
#Inicie el aumento de incrustaciones de palabras contextuales (BERT, DistilBERT, RoBERTA o XLNet)
aug_emb = naw.ContextualWordEmbsAug(
  # Other models include 'distilbert-base-uncased', 'roberta-base', etc.
  # Otros modelos incluyen 'distilbert-base-uncased', 'roberta-base', etc.
  model_path = 'distilbert-base-uncased',
  # You can also choose "insert" - También puede elegir "insertar"
  action = "substitute",
  # Use GPU
  device = 'cuda'
  )

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:72: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

In [32]:
# Initiate the back translation augmentation - Iniciar el aumento de traducción inversa
aug_bt = naw.BackTranslationAug(
  # Translate English to German -  Traducir inglés a alemán
  from_model_name = 'facebook/wmt19-en-de',
  # Translate German back to English - Traducir alemán de nuevo a inglés
  to_model_name = 'facebook/wmt19-de-en',
  # Use GPU
  device = 'cuda')

config.json:   0%|          | 0.00/825 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.08G [00:00<?, ?B/s]

Some weights of FSMTForConditionalGeneration were not initialized from the model checkpoint at facebook/wmt19-en-de and are newly initialized: ['model.encoder.embed_positions.weight', 'model.decoder.embed_positions.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


generation_config.json:   0%|          | 0.00/235 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/825 [00:00<?, ?B/s]

...

vocab-src.json:   0%|          | 0.00/849k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/315k [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/67.0 [00:00<?, ?B/s]

vocab-src.json:   0%|          | 0.00/849k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/315k [00:00<?, ?B/s]

In [33]:
# Create a function to evaluate text augmentation on model performance on test set
# Cree una función para evaluar el aumento de texto en el rendimiento del modelo en el conjunto de prueba
def evaluate_aug(aug_strategy, n, train_df, dset_test):

  # Create two lists to store augmented tweets and their correponding labels
  # Cree dos listas para almacenar tweets aumentados y sus etiquetas correspondientes
  augmented_text = []
  augmented_archivo = []

  # Loop through the train set to create augmented tweets
  # Note: We create n augmented tweets per original tweet.
  # Recorre el juego de train para crear tweets aumentados
  # Nota: Creamos n tweets aumentados por tweet original.
  for i in train_df.index:
    if aug_strategy == 'synonym':
      lst_augment = aug_syn.augment(train_df['text'].loc[i], n = n)
    elif aug_strategy == 'embedding':
      lst_augment = aug_emb.augment(train_df['text'].loc[i], n = n)
    else:
      lst_augment = aug_bt.augment(train_df['text'].loc[i], n = n)
    for augment in lst_augment:
      augmented_text.append(augment)
      augmented_archivo.append(train_df['archivo'].loc[i])

  # Zip the two lists into a list of tuples -  Comprima las dos listas en una lista de tuplas
  augmented_textarchivo = list(zip(augmented_text, augmented_archivo))

  # Convert the list of tuples to a Pandas Dataframe. -  Convierte la lista de tuplas en un marco de datos de Pandas.
  df_augmented_text = pd.DataFrame(
    augmented_textarchivo, columns = ['text', 'archivo'])

  # Vertically concat the train set with the augmented texts
  # Conectar verticalmente el set de train con los textos aumentados
##train_df_augmented = pd.concat([train_df, df_augmented_text], axis = 0)
  # Convert the DataFrame to a Dataset (Aparche Arrow format)
  # Convertir el marco de datos en un conjunto de datos
##dset_train_augmented = Dataset.from_pandas(train_df_augmented)
 ##dset_train_augmented = dset_train_augmented.remove_columns(["__index_level_0__"])
  # Gather train, val, and test Datasets to have a single DatasetDict,
  # which can be manipulated together
  # Reúna conjuntos de datos de train, val, y test para tener un solo DatasetDict,
  # que se pueden manipular juntos
##text_augmented = DatasetDict({
##    'train': dset_train_augmented,
##    'test': dset_test})

  return(df_augmented_text)

In [34]:
# Evaluar el aumento de texto sinónimo
DataFrame_synonym = evaluate_aug(
  aug_strategy = 'synonym',
  n = 1,
  train_df = train_df,
  dset_test = dset_test)
print(DataFrame_synonym)

                                                  text           archivo
0    AIK HUAT ironware ENTERPRISE (SETIA ALAM) SDN ...  X51007339642.txt
1    UNIHAKKA INTERNATIONAL SDN BHD 10 MAR 2018 18:...  X51005433522.txt
2    DIGI TELECOMMUNICATIONS SDN BHD (201283 - M) L...  X51006555072.txt
3    MR. D. I. Y. (M) SDN BHD (CO. REG: 860671 - D)...  X51007103675.txt
4    HOME MASTER HARDWARE & ELECTRICAL NO. 113G & 1...  X51005301659.txt
..                                                 ...               ...
621  99 SPEED MART S / B (519537 - X) LOT P. T. 281...  X51005719883.txt
622  ZEN AND COFFEE 28 S / B NO 53, JALAN 3 / 62A, ...  X51005719906.txt
623  UNIHAKKA INTERNATIONAL SDN BHD 10 MAY 2018 12:...  X51006913030.txt
624  FARMASI MALURI S / B (587969 - U) GST NO: 0003...  X51005685357.txt
625  UNIHAKKA INTERNATIONAL SDN BHD 12 JUN 2018 18:...  X51007846307.txt

[626 rows x 2 columns]


In [35]:
DataFrame_synonym.describe()

,text,archivo
count,626,626
unique,626,626
top,AIK HUAT ironware ENTERPRISE (SETIA ALAM) SDN ...,X51007339642.txt
freq,1,1


In [36]:
# Evaluate the embedding text augmentation
# Evaluar el aumento de texto incrustado
DataFrame_embedding = evaluate_aug(
  aug_strategy = 'embedding',
  n = 1,
  train_df = DataFrame_synonym,
  dset_test = dset_test)
print(DataFrame_embedding)

                                                  text           archivo
0    aik huat ironware enterprise ( setia alam ) sd...  X51007339642.txt
1    unihakka international sdn commenced 10 mar 20...  X51005433522.txt
2    digi telecommunications sdn bhd ( u - m ) mala...  X51006555072.txt
3    mr. d. i. y. ( m ) sdn bhd ( co. reg : 860671 ...  X51007103675.txt
4    malay master hardware & electrical no. 113g & ...  X51005301659.txt
..                                                 ...               ...
621  99 speed mart 6 / b ( 519537 - x ) lot p. t. 2...  X51005719883.txt
622  zen and coffee 28 s / b no 53, jalan 3 / 62a, ...  X51005719906.txt
623  unihakka international sdn bhd 10 may 2018 12 ...  X51006913030.txt
624  farmasi maluri s / b ( 587969 - u ) gst no : 0...  X51005685357.txt
625  unihakka international sdn tuesday 12 jun 2018...  X51007846307.txt

[626 rows x 2 columns]


In [37]:
DataFrame_embedding.describe()

,text,archivo
count,626,626
unique,626,626
top,aik huat ironware enterprise ( setia alam ) sd...,X51007339642.txt
freq,1,1


In [38]:
train_df_augmented = pd.concat([train_df, DataFrame_embedding], axis = 0)

In [39]:
train_df_augmented.describe()

,text,archivo
count,1252,1252
unique,1250,626
top,B & BEST RESTAURANT (001610235-P) GST NO. : 00...,X51007339642.txt
freq,2,2


In [40]:
# Evaluate the back translation text augmentation
# Evaluar el aumento de texto de traducción inversa
DataFrame_backtranslation = evaluate_aug(
  aug_strategy = 'backtranslation',
  n = 1,
  train_df = DataFrame_embedding,
  dset_test = dset_test)
print(DataFrame_backtranslation)

                                                  text           archivo
0    aik huat ironware enterprise (setia alam) sdn ...  X51007339642.txt
1    unihakka international sdn started March 10, 2...  X51005433522.txt
2    digi telecommunications sdn bhd (u - m) malays...  X51006555072.txt
3    mr. d. i. J. (m) sdn bhd (Co.-Nr.: 860671 - d)...  X51007103675.txt
4    Malaysian Master Hardware & Electrics No. 113g...  X51005301659.txt
..                                                 ...               ...
621  99 speed mart 6 / b (519537 - x) lot p. t. 281...  X51005719883.txt
622  Zen and Coffee 28 s / b no 53, jalan 3 / 62a, ...  X51005719906.txt
623  unihakka international sdn bhd May 10, 2018 12...  X51006913030.txt
624  farmasi maluri s / b (587969 - u) gst no: 0003...  X51005685357.txt
625  unihakka international sdn Tuesday, June 12, 2...  X51007846307.txt

[626 rows x 2 columns]


In [41]:
DataFrame_backtranslation.describe()

,text,archivo
count,626,626
unique,626,626
top,aik huat ironware enterprise (setia alam) sdn ...,X51007339642.txt
freq,1,1


In [42]:
train_df_augmented = pd.concat([train_df, DataFrame_backtranslation], axis = 0)
train_df_augmented.describe()

,text,archivo
count,1252,1252
unique,1250,626
top,B & BEST RESTAURANT (001610235-P) GST NO. : 00...,X51007339642.txt
freq,2,2


In [43]:
train_df_augmented.to_parquet('Sorieaumentation.parquet', index=False)